In [1]:
# !pip install modin
# !pip install modin[dask]
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision

In [0]:
!rm *

In [0]:
from google.colab import files
a = files.upload()

In [0]:
b = files.upload()

In [0]:
import io
mnist_train = pd.read_csv(io.BytesIO(a['mnist_train.csv']))
mnist_test = pd.read_csv(io.BytesIO(b['mnist_test.csv']))

In [0]:
mnist_train.head()

In [0]:
mnist_train = mnist_train.dropna()
mnist_test = mnist_test.dropna()

In [0]:
random_sel = mnist_train.sample(8)
random_sel.shape

In [0]:
image_feature = random_sel.drop('label', axis=1)
image_batch = (torch.Tensor(image_feature.values / 255.)).reshape((-1, 28, 28))
image_batch.shape

In [0]:
grid = torchvision.utils.make_grid(image_batch.unsqueeze(1), nrow=8)
grid.shape

In [0]:
plt.figure(figsize=(12,12))
plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.axis('off')

In [0]:
mnist_train_features = mnist_train.drop('label', axis=1)
mnist_train_target = mnist_train['label']

mnist_test_features = mnist_test.drop('label', axis=1)
mnist_test_target = mnist_test['label']

In [0]:
X_train_tensor = torch.tensor(mnist_train_features.values, dtype=torch.float)
x_test_tensor = torch.tensor(mnist_test_features.value, dtype=torch.float)

Y_train_tensor = torch.tensor(mnist_train_target.values, dtype=torch.long)
y_test_tensor = torch.tensor(mnist_test_target.values, dtype=torch.log)

In [0]:
X_train_tensor.shape, Y_train_tensor.shape, x_test_tensor.shape, y_test_tensor.shape

In [0]:
X_train_tensor = X_train_tensor.reshape(-1, 1, 28, 28)
x_test_tensor = X_test_tensor.reshape(-1, 1, 28, 28)

In [0]:
X_train_tensor.shape, Y_train_tensor.shape, x_test_tensor.shape, y_test_tensor.shape

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
#number of channels in the input, for grayscale images this will be 1
in_size = 1
hid1_size = 16
hid2_size = 32
#We have ten possible digits as our output
out_size = 10
#5x5 kernel
k_conv_size = 5

In [0]:
class ConvNet(nn.Module):

  def __init__(self):
    super(ConvNet, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(in_size, hid1_size, k_conv_size),
        nn.BatchNorm2d(hid1_size),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(hid1_size, hid2_size, k_conv_size),
        nn.BatchNorm2d(hid2_size),
        nn.ReLU(),
        nn.MaxPool2d(kernal_size=2)
    )
    self.fc = nn.Linear(512, out_size)

    def forward(self, x):

      out = self.layer1(x)
      print(out.shape)

      out = self.layer2(out)
      print(out.shape)

      out = out.reshape(out.size(0), -1)
      print(out.shape)

      out = self.fc(out)
      print(out.shape)

      return out

In [0]:
model = ConvNet()

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:
model.to(device)

In [0]:
X_train_tensor = X_train_tensor.to(device)
x_test_tensor = x_test_tensor.to(device)

Y_train_tensor = Y_train_tensor.to(device)
y_test_tensor = y_test_tensor.to(device)

In [0]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [0]:
num_epochs = 10
loss_values = list()

In [0]:
for epoch in range(1, num_epochs):

  outputs = model(X_train_tensor)
  loss = criterion(outputs, Y_train_tensor)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  print('Epoch - %d, loss - %0.5f' %(epoch, loss.item()))
  loss_values.append(loss.item())

In [0]:
x = (range(0,9))

plt.figure(figsize=(8,8))
plt.plot(x, loss_values)
plt.xlabel('Epoch')
plt.ylabel('Loss')

In [0]:
model.eval()

In [0]:
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [0]:
with torch.no_grad():

  correct = 0
  total = 0

  outputs = model(x_test_tensor)
  _, predicted = torch.max(outputs.data, 1)

  y_test = y_test_tensor.cpu().numpy()
  predicted = predicted.cpu()

  print("Accuracy: ", accuracy_score(predicted, y_test))
  print("Precision: ", precision_score(predicted, y_test, average='weighted'))
  print("Recall: ", recall_score(predicted, y_test, average='weighted'))

In [0]:
#number of channels in the input, for grayscale images this will be 1
in_size = 1
hid1_size = 16
hid2_size = 32
#We have ten possible digits as our output
out_size = 10
#5x5 kernel
k_conv_size = 5
learning_rate = 0.01
criterion = nn.NLLLoss()

#change forward return to F.log_softmax(out, dim=-1)